<a href="https://colab.research.google.com/github/dimicorn/bubbles/blob/master/bubble_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# All values of gamma
G = [1.001, 1.01, 1.4, 1.5, 1.6, 5/3, 1.7, 1.9, 2, 4]  # 1.05, 1.1, 1.2, 1.3, 1.4

# Same for k_rho
K = [0, 0.5, 1, 1.5, 2, 2.5, 3]

# And n_int too
N_int = [0, 0.5, 1, 1.5, 2, 2.5, 3]

# Colors for plot option
Colors = ['purple', 'blue', 'red', 'green', 'cyan', 'orange', 'black', 'gray']

# Line styles for different n_int
Line_style = ['solid', 'dashed', 'dashdot', 'dotted']

# Starting and ending values of lambda
x0 = 1
x_k = 0.7

# Initial conditions
v0 = 1
rho0 = 1
p0 = 1

# Plot labels
Gamma = 'gamma = '
K_rho = 'k_rho = '
N = 'n_int = '

# The place where will be created an excel table with all the data
file_path = "C:/Users/dmitr/Desktop/MIPT/Лето 21/Dr. Shang/Bubbles/Data/Auto2.xlsx"

au = 150 * 10**9

# For data option starting values of the parameters are being used
# Starting and ending values of velocity
v_int = 1000
vk = 100000

# Starting and ending number of years for time
n0 = 1000
nk = 100000

# Starting and ending values of ratio R_c/R_sw
k0 = 10
k_k = 1000

from scipy import integrate
import numpy as np


class Bubble(object):
    def __init__(self, a, b, c):
        self.gamma = a
        self.k_rho = b
        self.n_int = c
        g = self.gamma
        k = self.k_rho

        # Solving system of ODE's
        def solver(x, r):
            v, rho, p = r
            n1 = self.n_int
            n = (2 + n1) / (5 - k)
            fv = (4 * g * v / ((g + 1) * x) - k - (1 - 1 / n) * (
                    ((g + 1) / (g - 1)) * (2 * v / (g + 1) - x) * rho * v / p - 2)) * (
                         ((g + 1) / (g - 1)) * (2 * v / (g + 1) - x) ** 2 * rho / p - 2 * g / (g + 1)) ** (-1)
            fp = ((1 / n - 1) * v - (2 * v / (g + 1) - x) * fv) * ((g + 1) / ((g - 1) * rho))
            f_rho = ((-k * (g - 1) - 2 * (1 - 1 / n)) * (2 * v / (g + 1) - x) ** (-1) - 1 / p * fp) * (-rho / g)
            return fv, f_rho, fp

        # Solutions for the system
        sol = integrate.solve_ivp(solver, (x0, x_k), (v0, rho0, p0),
                                  t_eval=np.linspace(x0, x_k, 1000), method='Radau')
        self.velocity, self.density, self.pressure = sol.y
        self.lambda_c = sol.t
        self.slope_x = np.linspace(x0, x_k, 1000)
        self.slope_y = np.array([(self.gamma + 1) / 2 * self.slope_x[e] for e in range(1000)])
        self.dist = -1
        self.intersect = False
        x1, y1, x2, y2 = self.lambda_c, self.velocity, self.slope_x, self.slope_y
        count = 0
        for i in range(len(x1)):
            for j in range(len(x2)):
                if x1[i] == x2[j] and y1[i] == y2[j]:
                    count += 1
                    self.dist = 0
        if count == 0:
            lamb, vel = self.lambda_c[-1], self.velocity[-1]
            s = (lamb - 2 / (g + 1) * vel) * (2 * (g + 1)) / (4 + (g + 1) ** 2)
            ox = lamb - s * (g + 1) / 2
            oy = ox * (g + 1) / 2
            d = np.sqrt((ox - lamb) ** 2 + (oy - vel) ** 2)
            self.dist = d
        if self.dist < 0.001:
            self.intersect = True

        """
        # Searching for intersection
        def intersect():
            x1, y1, x2, y2 = self.lambda_c, self.velocity, self.slope_x, self.slope_y
            count = 0
            for i in range(len(x1)):
                for j in range(len(x2)):
                    if x1[i] == x2[j] and y1[i] == y2[j]:
                        count += 1
                        return True
            if count == 0:
                return False

        # Calculating the distance between the last point and the slope
        def d():
            if self.intersect():
                return 0
            else:
                g = self.gamma
                x, y = self.lambda_c[-1], self.velocity[-1]
                k = (x - 2 / (g + 1) * y) * (2 * (g + 1)) / (4 + (g + 1) ** 2)
                ox = x - k * (g + 1) / 2
                oy = ox * (g + 1) / 2
                r = np.sqrt((ox - x) ** 2 + (oy - y) ** 2)
                return r
        """

    # Function for eta
    def eta(self):
        k_rho = self.k_rho
        n_int = self.n_int
        x = (2 + n_int) / (5 - k_rho)
        return x

    # Value of the curve at lambda_c
    def v2(self, x, y):
        z = (x + 1) / 2 * y
        return z

    # Approximation using eqn B8a
    def lambda2(self, x, k_rho):
        w = self.eta()
        t = x ** 3 + 12 * x ** 2 + 8 * x + 1 - 0.5 * (x + 1) * (3 * x + 1) * k_rho - (x + 1) * (4 * x + 1) / w
        u = 2 * x ** 3 + 12 * x ** 2 + 7 * x + 1 - 0.5 * (x + 1) * (3 * x + 1) * k_rho - (x + 1) * (4 * x + 1) / w
        return t / u

    # Gradient of velocity, r = R_s
    def dv1(self, x, k_rho):
        w = self.eta()
        y = (-(7 * x + 3) + (x + 1) * k_rho + 3 * (x + 1) / w) / (x + 1)
        return y

    # Gradient of velocity, r = R_c
    def dv2(self, x, k_rho):
        w = self.eta()
        y = (-2 * (x + 1) + k_rho + 2 / w) / (2 * x / (x + 1))
        return y

    # Difference between lambda from the ODE and the approximation using eqn B8a
    def delta(self, x, y):
        z = abs(x - y) / x * 100
        return z

    # Calculating R_sw
    def r_sw(self, v, numb, k):
        g = self.gamma
        f_rho = (4 * g / (g + 1) ** 2) ** (1 / (g - 1))
        t_y = 3600 * 24 * 365
        t = numb * t_y
        n = (2 + self.n_int) / (5 - self.k_rho)
        r = (g - 1) / (g + 1) * v * f_rho * t * (3 * g / (3 * (g - 1) * n + self.n_int)) / k ** 3
        return r / au

    # Adding all the necessary data to the excel table
    def values(self, sh1, count):
        d = self.dist
        if self.intersect:
            sh1['A' + str(count)].value = self.gamma
            sh1['B' + str(count)].value = self.k_rho
            sh1['C' + str(count)].value = self.n_int
            l2 = self.lambda2(self.gamma, self.k_rho)
            sh1['D' + str(count)].value = self.delta(self.lambda_c[-1], l2)
            sh1['E' + str(count)].value = d
            sh1['F' + str(count)].value = self.q_p()

            # This works
            '''
            sh1['D' + str(count)].value = self.lambda_c[-1]
            sh1['E' + str(count)].value = self.velocity[-1]
            sh1['F' + str(count)].value = self.v2(self.gamma, self.lambda_c[-1])
            sh1['G' + str(count)].value = l2
            sh1['J' + str(count)].value = self.dv1(self.gamma, self.k_rho)
            sh1['K' + str(count)].value = self.dv2(self.gamma, self.k_rho)
            sh1['L' + str(count)].value = self.r_sw(v_int, n0, k0)
            sh1['F' + str(count)].value = self.p_r()
            sh1['G' + str(count)].value = self.p_sw()
            '''

    # Scaling for the velocity curves
    def norm1(self):
        lamb, vel, pres = self.lambda_c, self.velocity, self.pressure
        for t in range(len(lamb)):
            lamb[t] = 1 - (1 - lamb[t]) / (self.gamma - 1)
        for t in range(len(vel)):
            vel[t] = 1 + (vel[t] - 1) / (self.gamma - 1)
        return lamb, vel

    # Scaling for the linear slope
    def norm2(self):
        g = self.gamma
        a = self.slope_x
        b = self.slope_y
        for t in range(len(a)):
            a[t] = 1 - (1 - a[t]) / (g - 1)
        for t in range(len(b)):
            b[t] = 1 + (b[t] - 1) / (g - 1)
        return a, b

    # Values of Q_p
    def q_p(self):
        g = self.gamma
        p_r = 2 / (g + 1) * self.pressure[-1]
        p_sw = (2/(g + 1)) * ((g + 1)**2/(4*g))**(g/(g-1))
        return p_sw/p_r

import matplotlib.pyplot as plt

# Repeating plot legend for pressure
def leg():
    plt.plot([], [], 'k-', label=N + str(N_int[0]))
    plt.plot([], [], 'k--', label=N + str(N_int[1]))
    plt.plot([], [], 'k-.', label=N + str(N_int[2]))
    plt.plot([], [], 'k.', label=N + str(N_int[3]))


# Repeating plot label
def lab(y):
    plt.grid()
    plt.xlabel('Lambda')
    plt.ylabel('P')
    plt.title('P(Lambda), ' + Gamma + str(G[y]))
    plt.legend()
    plt.savefig('P(Lambda), ' + Gamma + str(G[y]) + '.png', dpi=300)
    plt.show()

import openpyxl

K_ext = np.linspace(0, 3, num=10)
N_ext = np.linspace(0, 3, num=10)


def main():
    a = str(input('Data or plot? '))
    c = ''
    d = ''
    if a == 'plot':
        b = str(input('Scatter or no? '))
        if b == 'scat':
            d = str(input('3d or 2d? '))
        elif b == 'no':
            c = str(input('Velocity or pressure? '))

    if a == 'data':
        count = 2
        wb = openpyxl.Workbook()
        wb['Sheet'].title = 'Sheet1'
        sh1 = wb.active
        sh1['A1'].value = 'gamma'
        sh1['B1'].value = 'k_rho'
        sh1['C1'].value = 'n_int'
        sh1['D1'].value = 'distance'
        sh1['E1'].value = 'Q_p'

    for i in range(len(G)):
        if d == '3d':
            fig = plt.figure()
            ax = plt.axes(projection='3d')
        for j in range(len(K_ext)):
            '''
            if c == 'pres':
                count = 0
            '''
            for q in range(len(N_ext)):
                bubble = Bubble(G[i], K_ext[j], N_ext[q])
                if a == 'data' and bubble.intersect:
                    bubble.values(sh1, count)
                    count += 1
                elif d == '3d' and bubble.intersect:
                    q_p = bubble.q_p()
                    if 0.9 < q_p < 1.15:
                        ax.scatter3D(K[j], N_int[q], bubble.q_p(), c='blue', alpha=0.5)  # , c=zdata, cmap='Greens'
                    else:
                        ax.scatter3D(K[j], N_int[q], bubble.q_p(), c='red', alpha=0.5)
                elif d == '2d':
                    if bubble.intersect:
                        q_p = bubble.q_p()
                        if 0.9 < q_p < 0.95:
                            plt.scatter(K_ext[j], N_ext[q], c='cyan')
                        elif 0.95 < q_p < 1.05:
                            plt.scatter(K_ext[j], N_ext[q], c='blue')
                        elif 1.05 < q_p < 1.15:
                            plt.scatter(K_ext[j], N_ext[q], c='darkcyan')
                        else:
                            plt.scatter(K_ext[j], N_ext[q], c='red')
                    else:
                        plt.scatter(K_ext[j], N_ext[q], c='gray')
                elif c == 'vel':
                    lamb, vel = bubble.lambda_c, bubble.velocity
                    plt.plot(lamb, vel, label=N + str(N_int[q]))
                    if q == 0:
                        plt.plot(bubble.slope_x, bubble.slope_y, label=Gamma + str(G[i]))
                '''
                elif c == 'pres' and bubble.intersect:
                    lamb, pres = bubble.lambda_c, bubble.pressure
                    if count == 0:
                        plt.plot(lamb, pres, color=Colors[j], label=K_rho + str(K[j]), linestyle=Line_style[q])
                        count += 1
                    else:
                        plt.plot(lamb, pres, color=Colors[j], linestyle=Line_style[q])
                '''
            if c == 'vel':
                if i == 0:
                    plt.axis([0.99, 1., 0.99, 1.02])
                elif i == 1:
                    plt.axis([0.95, 1., 0.9, 1.2])
                elif i == 2:
                    plt.axis([0.9, 1., 0.9, 1.2])
                elif i == 3:
                    plt.axis([0.825, 1., 0.9, 1.2])
                elif i == 4:
                    plt.axis([0.75, 1., 0.9, 1.3])
                plt.grid()
                plt.legend()
                plt.xlabel('Lambda')
                plt.ylabel('V')
                plt.title('V(Lambda), ' + Gamma + str(G[i]) + ', ' + K_rho + str(K[j]))
                plt.savefig('V(Lambda), ' + Gamma + str(G[i]) + ', ' + K_rho + str(K[j]) + '.png', dpi=300)
                plt.show()

        if d == '3d':
            plt.title('Q_p, ' + Gamma + str(G[i]))
            ax.set_xlabel('K_rho')
            ax.set_ylabel('N_int')
            ax.set_zlabel('Q_p')
            fig.savefig('Q_p, ' + Gamma + str(G[i]) + '.png', dpi=300)
            fig.show()
        elif d == '2d':
            plt.plot([], [], c='cyan', marker='o', label='0.9 < Q_p < 0.95')
            plt.plot([], [], c='blue', marker='o', label='0.95 < Q_p < 1.05')
            plt.plot([], [], c='darkcyan', marker='o', label='1.05 < Q_p < 1.15')
            plt.title('Colormap, ' + Gamma + str(G[i]))
            plt.xlabel('K_rho')
            plt.ylabel('N_int')
            plt.legend()
            plt.savefig('Colormap, ' + Gamma + str(G[i]) + '.png', dpi=300)
            plt.show()
        '''
        elif c == 'pres':
            leg()
            lab(i)
        '''
    if a == 'data':
        wb.save(file_path)